### Webscrapping

In [45]:
import time
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib.request as urllib

import pandas as pd

In [2]:
# setup
DRIVER_PATH = '/webdrivers/chromedriver.exe'
browser = webdriver.Chrome(executable_path=DRIVER_PATH)

dict_of_link = {
    '2011-12Season':'https://www.premierleague.com/results?co=1&se=19&cl=-1'   
}

def get_browser_data(dict_of_link):
    for season, season_link in dict_of_link.items():
        # Get the the premier league website at a specific year
        browser.get(season_link)
        time.sleep(2)

        # Get the body of the site
        element_body = browser.find_element_by_tag_name("body")
        # Scroll
        no_of_pagedowns = 50

        # get the list of match id for stats data
        match_id_lst = []

        for go_down in range(50,0,-1):
            element_body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)

        unordered_list_matchList = browser.find_elements_by_class_name("matchList")

        for matchList_container in unordered_list_matchList:
            li_element = matchList_container.find_elements_by_class_name("matchFixtureContainer")
            for id_attr in li_element:
                match_id = id_attr.get_attribute("data-comp-match-item")
                match_id_lst.append(match_id)
        return match_id_lst

In [3]:
def get_more_data(match_id_lst):
    # loop through match ids and open the actual match in details
    all_stat_dict = {}
    
    for match_id in match_id_lst:
        #full url
        browser.get("https://www.premierleague.com/match/" + match_id)
        time.sleep(1)
        tabs = browser.find_element_by_tag_name("body")
        stat_tab = tabs.find_element_by_css_selector("div.tabLinks.matchNav > .tabs ul li:nth-child(3)")
        stat_tab.click()
        time.sleep(1)

        stat_table = browser.find_element_by_class_name("matchCentreStatsContainer")
        table_len = len(stat_table.find_elements_by_xpath("./tr"))

        count = 1
        hometeam_name = browser.find_element_by_xpath("""//*[@id="mainContent"]/div/section[2]/div[2]/div/div[2]/section[3]/div[2]/div[2]/table/thead/tr/th[1]/a
        """).text
        awayteam_name = browser.find_element_by_xpath("""//*[@id="mainContent"]/div/section[2]/div[2]/div/div[2]/section[3]/div[2]/div[2]/table/thead/tr/th[3]/a""").text

        fullTime_score = browser.find_element_by_xpath("""//*[@id="mainContent"]/div/section[2]/div[2]/section/div[3]/div/div/div[1]/div[2]/div/div""").text.split('-')
        hometeam_FTS = fullTime_score[0]
        awayteam_FTS = fullTime_score[1]

        hometeam_pos='0'
        awayteam_pos='0'
        hometeam_sot='0'
        awayteam_sot='0'
        hometeam_shots='0'
        awayteam_shots='0',
        hometeam_touches='0'
        awayteam_touches='0'
        hometeam_tackles='0'
        awayteam_tackles='0'
        hometeam_clearances='0'
        awayteam_clearances='0'
        hometeam_corners='0'
        awayteam_corners='0'
        hometeam_offsides='0'
        awayteam_offsides='0'
        hometeam_yCards='0'
        awayteam_yCards='0'
        hometeam_rCards='0'
        awayteam_rCards='0'
        hometeam_fConceded='0'
        awayteam_fConceded='0'


        for stat_row in range(table_len):
            category = stat_table.find_element_by_xpath("""./tr[{}]/td[2]/p""".format(count))
            hometeam = stat_table.find_element_by_xpath("""./tr[{}]/td[1]/p""".format(count))
            awayteam = stat_table.find_element_by_xpath("""./tr[{}]/td[3]/p""".format(count))
            count +=1

            if "Possession" in category.text:
                hometeam_pos = hometeam.text
                awayteam_pos = awayteam.text
            elif "Shots on target" in category.text:
                hometeam_sot = hometeam.text
                awayteam_sot = awayteam.text
            elif "Shots" in category.text:
                hometeam_shots = hometeam.text
                awayteam_shots = awayteam.text
            elif "Touches" in category.text:
                hometeam_touches = hometeam.text
                awayteam_touches = awayteam.text
            elif "Passes" in category.text:
                hometeam_touches = hometeam.text
                awayteam_touches = awayteam.text
            elif "Tackles" in category.text:
                hometeam_tackles = hometeam.text
                awayteam_tackles = awayteam.text
            elif "Clearance" in category.text:
                hometeam_clearances = hometeam.text
                awayteam_clearances = awayteam.text
            elif "Corners" in category.text:
                hometeam_corners = hometeam.text
                awayteam_corners = awayteam.text
            elif "Offsides" in category.text:
                hometeam_offsides = hometeam.text
                awayteam_offsides = awayteam.text
            elif "Yellow cards" in category.text:
                hometeam_yCards = hometeam.text
                awayteam_yCards = awayteam.text
            elif "Red cards" in category.text:
                hometeam_rCards = hometeam.text
                awayteam_rCards = awayteam.text
            elif "Fouls conceded" in category.text:
                hometeam_fConceded = hometeam.text
                awayteam_fConceded = awayteam.text

            #dictionary of matches with details for a specific season
            all_stat_dict["Match"+str(match_id)] = {"Match_id":match_id,
                                                    "Hometeam": hometeam_name,
                                                    "Awayteam":awayteam_name,
                                                    "Hometeam_Score":hometeam_FTS,
                                                    "Awayteam_Score":awayteam_FTS,
                                                    "Home_Possession":hometeam_pos,
                                                    "Away_Possession":awayteam_pos,
                                                    "Home_Shots_on_target":hometeam_sot,
                                                    "Away_Shots_on_target":awayteam_sot,
                                                    "Home_Total_Shots":hometeam_shots,
                                                    "Away_Total_Shots":awayteam_shots,
                                                    "Home_Touches":hometeam_touches,
                                                    "Away_Touches":awayteam_touches,
                                                    "Home_Passes":hometeam_touches,
                                                    "Away_Pasees":awayteam_touches,
                                                    "Home_Takles":hometeam_tackles,
                                                    "Away_Takles":awayteam_tackles,
                                                    "Home_Clearance":hometeam_clearances,
                                                    "Away_Clearance":awayteam_clearances,
                                                    "Home_Corners":hometeam_corners,
                                                    "Away_Corners":awayteam_corners,
                                                    "Home_Offsides":hometeam_offsides,
                                                    "Away_Offsides":awayteam_offsides,
                                                    "Home_Yellow_Cards":hometeam_yCards,
                                                    "Away_Yellow_Cards":awayteam_yCards,
                                                    "Home_Red_Cards":hometeam_rCards,
                                                    "Away_Red_Cards":awayteam_rCards,
                                                    "Home_Fouls_Conceded":hometeam_fConceded,
                                                    "Away_Fouls_Conceded":awayteam_fConceded 
                                                    }
    return all_stat_dict

In [4]:
def write_to_csv(stat_dict, filename):
    with open(filename+'.csv', 'w') as f:
        f.write("Match_ID" + "," + "Hometeam" + "," + "Awayteam" + "," + "Hometeam Score" + "," + "Awayteam Score" + "," + "Home_Possession" 
                + "," + "Away_Possession" + "," + "Home_Shots_on_target" + "," + "Away_Shots_on_target" 
                + "," + "Home_Total_Shots"+ "," + "Away_Total_Shots" + "," + "Home_Touches" 
                + "," + "Away_Touches" + "," + "Home_Passes" + "," + "Away_Pasees" + "," + "Home_Takles"
                + "," + "Away_Takles"+ "," + "Home_Clearance"+ "," + "Away_Clearance"
                + "," + "Home_Corners"+ "," + "Away_Corners"+ "," + "Home_Offsides"
                + "," + "Away_Offsides"+ "," + "Home_Yellow_Cards"+ "," + "Away_Yellow_Cards"
                + "," + "Home_Red_Cards"+ "," + "Away_Red_Cards"+ "," + "Home_Fouls_Conceded"
                + "," + "Away_Fouls_Conceded" + '\n')
        
        for i in stat_dict.keys():
            output_str = str(stat_dict[i]['Match_id']) + "," + str(stat_dict[i]['Hometeam'])+ "," + str(stat_dict[i]['Awayteam']) + "," + str(stat_dict[i]['Hometeam_Score']) + "," + str(stat_dict[i]['Awayteam_Score']) + "," + str(stat_dict[i]['Home_Possession'])+ "," + str(stat_dict[i]['Away_Possession']) + ", " + str(stat_dict[i]['Home_Shots_on_target'])+ "," + str(stat_dict[i]['Away_Shots_on_target']) + "," + str(stat_dict[i]['Home_Total_Shots']) + "," + str(stat_dict[i]['Away_Total_Shots'])+ "," + str(stat_dict[i]['Home_Touches'])+ "," + str(stat_dict[i]['Away_Touches'])+ "," + str(stat_dict[i]['Home_Passes']) + "," + str(stat_dict[i]['Away_Pasees'])+ "," + str(stat_dict[i]['Home_Takles'])+ "," + str(stat_dict[i]['Away_Takles'])+ "," + str(stat_dict[i]['Home_Clearance'])+ "," + str(stat_dict[i]['Away_Clearance'])+ "," + str(stat_dict[i]['Home_Corners'])+ "," + str(stat_dict[i]['Away_Corners'])+ "," + str(stat_dict[i]['Home_Offsides'])+ "," + str(stat_dict[i]['Away_Offsides'])+ "," + str(stat_dict[i]['Home_Yellow_Cards'])+ "," + str(stat_dict[i]['Away_Yellow_Cards'])+ "," + str(stat_dict[i]['Home_Red_Cards']) + "," + str(stat_dict[i]['Away_Red_Cards'])+ "," + str(stat_dict[i]['Home_Fouls_Conceded']) + "," + str(stat_dict[i]['Away_Fouls_Conceded']) + '\n'
            f.write(output_str)

In [5]:
match_id_lst = get_browser_data(dict_of_link)
stats_dict = get_more_data(match_id_lst)
print(stats_dict)

{'Match7457': {'Match_id': '7457', 'Hometeam': 'Aston Villa', 'Awayteam': 'Liverpool', 'Hometeam_Score': '1', 'Awayteam_Score': '0', 'Home_Possession': '48.5', 'Away_Possession': '51.5', 'Home_Shots_on_target': '3', 'Away_Shots_on_target': '3', 'Home_Total_Shots': '9', 'Away_Total_Shots': '12', 'Home_Touches': '434', 'Away_Touches': '465', 'Home_Passes': '434', 'Away_Pasees': '465', 'Home_Takles': '23', 'Away_Takles': '21', 'Home_Clearance': '21', 'Away_Clearance': '11', 'Home_Corners': '4', 'Away_Corners': '5', 'Home_Offsides': '2', 'Away_Offsides': '5', 'Home_Yellow_Cards': '2', 'Away_Yellow_Cards': '2', 'Home_Red_Cards': '0', 'Away_Red_Cards': '0', 'Home_Fouls_Conceded': '14', 'Away_Fouls_Conceded': '11'}, 'Match7458': {'Match_id': '7458', 'Hometeam': 'Bolton Wanderers', 'Awayteam': 'Manchester City', 'Hometeam_Score': '0', 'Awayteam_Score': '2', 'Home_Possession': '40.5', 'Away_Possession': '59.5', 'Home_Shots_on_target': '2', 'Away_Shots_on_target': '7', 'Home_Total_Shots': '15', 

In [6]:
write_to_csv(stats_dict, '2011-12Season')

In [7]:
df = pd.DataFrame.from_dict(stats_dict, orient='index')
df

,Match_id,Hometeam,Awayteam,Hometeam_Score,Awayteam_Score,Home_Possession,Away_Possession,Home_Shots_on_target,Away_Shots_on_target,Home_Total_Shots,...,Home_Corners,Away_Corners,Home_Offsides,Away_Offsides,Home_Yellow_Cards,Away_Yellow_Cards,Home_Red_Cards,Away_Red_Cards,Home_Fouls_Conceded,Away_Fouls_Conceded
Match7457,7457,Aston Villa,Liverpool,1,0,48.5,51.5,3,3,9,...,4,5,2,5,2,2,0,0,14,11
Match7458,7458,Bolton Wanderers,Manchester City,0,2,40.5,59.5,2,7,15,...,6,4,0,1,1,1,1,0,13,11
Match7459,7459,Everton,Chelsea,1,0,36.1,63.9,2,5,12,...,6,5,2,5,2,2,1,0,12,16
Match7460,7460,Fulham,Arsenal,2,2,32.2,67.8,3,5,9,...,5,3,4,2,2,1,1,0,14,7
Match7461,7461,Manchester United,Blackpool,4,2,58.5,41.5,7,4,20,...,8,8,3,1,2,1,0,0,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Match7089,7089,Bolton Wanderers,Fulham,0,0,46.5,53.5,5,4,11,...,4,8,5,3,1,3,0,0,11,14
Match7091,7091,Sunderland,Birmingham City,2,2,44.1,55.9,2,6,9,...,3,5,2,1,2,3,1,0,15,10
Match7093,7093,Wigan Athletic,Blackpool,0,4,56.4,43.6,5,6,11,...,6,4,3,4,1,1,0,0,8,12
Match7094,7094,Wolverhampton Wanderers,Stoke City,2,1,61.5,38.5,4,4,15,...,5,5,0,3,0,2,0,0,19,16


In [56]:
DRIVER_PATH = '/webdrivers/chromedriver.exe'
browser = webdriver.Chrome(executable_path=DRIVER_PATH)

link = 'https://www.premierleague.com/clubs?se=363'
browser.get(link)
time.sleep(2)

element_div = browser.find_element_by_class_name('indexAllTime')
alltime_table = browser.find_element_by_class_name("allTimeDataContainer")
table_len = len(alltime_table.find_elements_by_xpath("./tr"))

for team_id in range(table_len):
    teamTr = browser.find_element_by_xpath("""//*[@id="mainContent"]/div[2]/div/div/div[3]/div/table/tbody/tr[{}]/td[1]/a""".format(team_id+1))
    print(teamTr.get_attribute('href'), team_id )
    link = teamTr.get_attribute('href')
    download_icon(link)
    time.sleep(2)
    browser.back()
    browser.close()
    
    browser = webdriver.Chrome(executable_path=DRIVER_PATH)
    link = 'https://www.premierleague.com/clubs?se=363'
    browser.get(link)
    time.sleep(2)
#     teamTr.click()
#     
#     img = driver.find_element_by_xpath("""//*[@id="mainContent"]/header/div[2]/div/div/div[1]/picture/img""")
#     src = img.get_attribute('src')
    
#     urllib.urlretrieve(src, "test.png")

# driver.close()
#     print(team_id)
# print(table_len)
# print(alltime_table.get_attribute('innerHTML'))

https://www.premierleague.com/clubs/1/Arsenal/overview 0
https://www.premierleague.com/clubs/2/Aston-Villa/overview 1
https://www.premierleague.com/clubs/30/Barnsley/overview 2
https://www.premierleague.com/clubs/35/Birmingham-City/overview 3
https://www.premierleague.com/clubs/3/Blackburn-Rovers/overview 4
https://www.premierleague.com/clubs/44/Blackpool/overview 5
https://www.premierleague.com/clubs/27/Bolton-Wanderers/overview 6
https://www.premierleague.com/clubs/127/Bournemouth/overview 7
https://www.premierleague.com/clubs/32/Bradford-City/overview 8
https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/overview 9
https://www.premierleague.com/clubs/43/Burnley/overview 10
https://www.premierleague.com/clubs/46/Cardiff-City/overview 11
https://www.premierleague.com/clubs/31/Charlton-Athletic/overview 12
https://www.premierleague.com/clubs/4/Chelsea/overview 13
https://www.premierleague.com/clubs/5/Coventry-City/overview 14
https://www.premierleague.com/clubs/6/Crystal-P

In [46]:
def download_icon(link):
    browser.get(link)
    img = browser.find_element_by_xpath("""//*[@id="mainContent"]/header/div[2]/div/div/div[1]/picture/img""")
    src = img.get_attribute('src')
    urllib.urlretrieve(src, link.split('/')[-2]+".png")